In [1]:
from utils import TrajectoryInformation, DummyDataLoader, TrajectoryDataPipeline
from collections import deque, defaultdict
import numpy as np

data_path = '/home/kimbring2/minerl_data/'
env_name = 'MineRLObtainDiamondDense-v0'
trajectory_name = 'v3_absolute_grape_changeling-6_37339-46767'

#item_list = ['log', 'planks', 'crafting_table', 'stick', 'wooden_pickaxe', 'cobblestone', 'coal', 
#             'stone_pickaxe', 'iron_ore', 'furnace', 'iron_ingot', 'iron_pickaxe', 'stone']
item_list = ['log']

inventory_key_list = ['coal', 'cobblestone', 'crafting_table', 'dirt', 'furnace', 'iron_axe', 'iron_ingot', 
                      'iron_ore', 'iron_pickaxe', 'log', 'planks', 'stick', 'stone', 'stone_axe', 
                      'stone_pickaxe', 'torch', 'wooden_axe', 'wooden_pickaxe', 'pov']
data = None
for trajectory_name in TrajectoryDataPipeline.get_trajectory_names(data_path + env_name):
    print("trajectory_name: " + str(trajectory_name))
    # trajectory_name: v3_absolute_grape_changeling-6_37339-46767
    
    trajectory_information = TrajectoryInformation(data_path + env_name + "/" + trajectory_name)
    #for chain in trajectory_information.chain:
    #    print("chain.name: ", chain.name)
    
    trajectory = TrajectoryDataPipeline.load_data(data_path + env_name + "/" + trajectory_name)
    sliced_trajectory = trajectory_information.slice_trajectory_by_item(trajectory)
    
    for item in sliced_trajectory:
        #print("item: " + str(item))
        
        if item not in item_list:
            continue
        
        item_index = item_list.index(item)
        #print("item_index: " + str(item_index))
            
        scale = len(item_list)
        item_layer = np.zeros([64, 64, scale], dtype=np.float32)
        item_layer[:, :, item_index] = 1.0
            
        data = DummyDataLoader(data=sliced_trajectory, items_to_add=[item])

        for s, a, r, s1, d in data.batch_iter(batch_size=1, num_epochs=1, seq_len=5000):
            d = np.array(d)
            d = d.astype(int)
            
            #print("a.keys(): " + str(a.keys()))
            #print("a['place']: " + str(a['place']))
            
            length = (s['pov'].shape)[0]
            #print("length: " + str(length))
            length_adding = 0
            #print("")
            for i in range(0, length):
                action = {key: value[i] for key, value in a.items()}
                for key, value in action.items():
                    if key != 'camera':
                        action[key] = value
                    else:
                        #print("value[0]: " + str(value[0]))
                        #print("value[1]: " + str(value[1]))

                        camera_threshols = (abs(value[0]) + abs(value[1])) / 2.0
                        if camera_threshols > 2.5:
                            if (value[1] < 0) and ( abs(value[0]) < abs(value[1]) ):
                                action[key] = [0, -5]
                            elif (value[1] > 0) and ( abs(value[0]) < abs(value[1]) ):
                                action[key] = [0, 5]
                            elif (value[0] < 0) and ( abs(value[0]) > abs(value[1]) ):
                                action[key] = [-5, 0]
                            elif (value[0] > 0) and ( abs(value[0]) > abs(value[1]) ):
                                action[key] = [5, 0]
                            else:
                                action[key] = [0, 0]
                        else:
                            action[key] = [0, 0]
                                
                #print("action: " + str(action))
                discrete_action = {key: value for key, value in action.items()}
                #print("discrete_action: " + str(discrete_action))
                    
                inventory_channel = np.zeros((64,64,1))
                if 'inventory' in s:
                    region_max_height = s['pov'][i].shape[0]
                    region_max_width = s['pov'][i].shape[1]
                    rs = 8

                    if min(region_max_height, region_max_width) < rs:
                        raise ValueError("'region_size' is too large.")

                    num_element_width = region_max_width // rs

                    inventory_channel = np.zeros(shape=list(s['pov'][i].shape[:-1]) + [1], dtype=s['pov'][i].dtype)
                    for key_idx, key in enumerate(inventory_key_list):
                        item_scaled = np.clip(1 - 1 / (s['inventory'][key][i] + 1),  # Inversed
                                                          0, 1)
                        item_channel = np.ones(shape=[rs, rs, 1], dtype=s['pov'][i].dtype) * item_scaled
                        width_low = (key_idx % num_element_width) * rs
                        height_low = (key_idx // num_element_width) * rs

                        if height_low + rs > region_max_height:
                            raise ValueError("Too many elements on 'inventory'. Please decrease 'region_size' of each component.")

                        inventory_channel[height_low:(height_low + rs), width_low:(width_low + rs), :] = item_channel

                obs = np.concatenate((s['pov'][i] / 255.0, inventory_channel), axis=2)
                obs = np.concatenate((obs, item_layer), axis=2)
                #print("obs.shape: " + str(obs.shape))
                
                action_camera_0 = a['camera'][i][0]
                action_camera_1 = a['camera'][i][1]
                action_attack = a['attack'][i]
                action_forward = a['forward'][i]
                action_jump = a['jump'][i]
                action_back = a['back'][i]
                action_left = a['left'][i]
                action_right = a['right'][i]
                action_sneak = a['sneak'][i]
                
                camera_threshols = (abs(action_camera_0) + abs(action_camera_1)) / 2.0
                if (camera_threshols > 2.5):
                    if ( (action_camera_1 < 0) & ( abs(action_camera_0) < abs(action_camera_1) ) ):
                        if (action_attack == 0):
                            action_index = 0
                        else:
                            action_index = 1
                    elif ( (action_camera_1 > 0) & ( abs(action_camera_0) < abs(action_camera_1) ) ):
                        if (action_attack == 0):
                            action_index = 2
                        else:
                            action_index = 3
                    elif ( (action_camera_0 < 0) & ( abs(action_camera_0) > abs(action_camera_1) ) ):
                        if (action_attack == 0):
                            action_index = 4
                        else:
                            action_index = 5
                    elif ( (action_camera_0 > 0) & ( abs(action_camera_0) > abs(action_camera_1) ) ):
                        if (action_attack == 0):
                            action_index = 6
                        else:
                            action_index = 7

                elif (action_forward == 1):
                    if (action_attack == 0):
                        action_index = 8
                    else:
                        action_index = 9
                elif (action_jump == 1):
                    if (action_attack == 0):
                        action_index = 10
                    else:
                        action_index = 11
                elif (action_back == 1):
                    if (action_attack == 0):
                        action_index = 12
                    else:
                        action_index = 13
                elif (action_left == 1):
                    if (action_attack == 0):
                        action_index = 14
                    else:
                        action_index = 15
                elif (action_right == 1):
                    if (action_attack == 0):
                        action_index = 16
                    else:
                        action_index = 17
                elif (action_sneak == 1):
                    if (action_attack == 0):
                        action_index = 18
                    else:
                        action_index = 19
                elif (action_attack == 1):
                    action_index = 20
                else:
                    #for key in a.keys():
                        #print("a[key][i]: ", a[key][i])
                        #if key != 'camera':
                        #    print('a[%s][%s]: %s' % (key, i, a[key][i]))
                        #else:
                            #print('a[%s][%s][0]: %d , a[%s][%s][1]: %d' % (key, i, a[key][i][0], 
                            #                                             key, i, a[key][i][1]))
                        
                    #print("")
                        
                    continue
                
                if (a['attack'][i] == 0 and a['back'][i] == 0 and a['camera'][i][0] == 0.0 and 
                    a['camera'][i][1] == 0.0 and a['forward'][i] == 0 and a['jump'][i] == 0 and 
                    a['left'][i] == 0 and a['right'][i] == 0 and a['sneak'][i] == 0):
                    #print("continue: ")
                    continue
     
                length_adding += 1
                        
            print("length: " + str(length))
            print("length_adding: ", length_adding)
            print("")

    #break
    print("")

/home/kimbring2/.local/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


trajectory_name: v3_absolute_grape_changeling-17_418-2303
length: 1524
length_adding:  1344


trajectory_name: v3_absolute_grape_changeling-6_37339-46767
length: 1580
length_adding:  1506


trajectory_name: v3_absolute_grape_changeling-9_4643-34456
length: 1150
length_adding:  1146


trajectory_name: v3_absolute_zucchini_basilisk-14_12791-21691


KeyboardInterrupt: 

In [16]:
from utils import TrajectoryInformation, create_nodes
import numpy as np

data_path = '/home/kimbring2/minerl_data/'
env_name = 'MineRLObtainDiamondDense-v0'
trajectory_name = 'v3_gleeful_grape_gnome-8_201-10945'

trajectory_information = TrajectoryInformation(data_path + env_name + "/" + trajectory_name)
final_chain = trajectory_information.to_old_chain_format(items=trajectory_information.chain, return_time_indexes=False)

nodes = create_nodes(final_chain)
for node in nodes:
    print("node.name: ", node.name)
    
print("len(nodes): ", len(nodes))

node.name:  log
node.name:  dirt
node.name:  log
node.name:  dirt
node.name:  planks
node.name:  crafting_table
node.name:  stick
node.name:  wooden_pickaxe
node.name:  planks
node.name:  wooden_axe
node.name:  planks
node.name:  stick
node.name:  crafting_table
node.name:  log
node.name:  dirt
node.name:  cobblestone
node.name:  stone_pickaxe
node.name:  crafting_table
node.name:  cobblestone
node.name:  iron_ore
node.name:  iron_ore
node.name:  coal
node.name:  torch
node.name:  furnace
node.name:  iron_ingot
node.name:  iron_ingot
node.name:  iron_pickaxe
len(nodes):  27


In [17]:
print("len(trajectory_information.chain): ", len(trajectory_information.chain))

for action in trajectory_information.chain[3]['actions']:
    if action['value'] != 'none':  
        print("action.key: ", action.key
        #print("value: ", action['value'])

SyntaxError: unexpected EOF while parsing (648985473.py, line 6)

In [18]:
node = nodes[1]

for crafting_action in node.crafting_agent.crafting_actions:
    if list(crafting_action.keys())[0] == 'craft':
        print("crafting_action: ", crafting_action)
        #print("list(crafting_action.keys()): ", list(crafting_action.keys()))
        #print("node.crafting_agent.crafting_actions: ", node.crafting_agent.crafting_actions)

#for i in range(0, 50):
#    print("node.crafting_agent.get_crafting_action(): ", node.crafting_agent.get_crafting_action())

crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting_action:  {'craft': 1}
crafting